In [1]:
import pandas as pd
import glob
import os
import re

Objective: Create an income, balancesheet, and cashflow file, each with tabs corresponding to their reporting periods:
    e.g. Company A, if it has three reports, would have one file containing three tabs.

Structure:

<li>Create a list of companies to iterate over.</li>
<li>Create a master file to write for each company</li>
<li>Find the list of files associated with each company.</li>
<li>Iterate over each file, appending each one to a new tab in the company's master file.</li>
Rinse & Repeat.

Formalized version:
<li>Outer loop, looping through the list of companies</li>
<li>Inner loop that loops through all the files for each company</li>
<li>In the inner loop, append each file to a new tab in the master file</li>

In [2]:
def generate_company_list():
    companies = []
    unique_list = []
    file_list = glob.glob("C:/Users/curt.beck/Downloads/Cognaize/*.xlsx")
    for file in file_list:
        match_obj = re.search("[A-Za-z]+\_\d+|[A-Za-z]+\s+[\&A-Za-z\s\_\d]+", file)
        if match_obj is not None:
            split_str = match_obj.group().split('_')
            split_str_item = split_str[0]
            companies.append(split_str_item)
    
    for company in companies:
        if company not in unique_list:
            unique_list.append(company)
    no_spaces = [company.strip() for company in unique_list]
    clean_list = [company for company in no_spaces if company]
    return clean_list

In [3]:
def filter_sheets(sheet_list, statement_type):
    if statement_type == 'Income Statement':
        filtered_list = [sheet for sheet in sheet_list if re.search('IS\s+\-\s+\d{1}', sheet)]
        if len(filtered_list) > 0:
            return filtered_list[0]
        else:
            return [] 
    elif statement_type == 'Balance Sheet':
        filtered_list = [sheet for sheet in sheet_list if re.findall('BS\s+\-\s+\d{1}', sheet)]
        if len(filtered_list) > 1:
            return filtered_list
        elif len(filtered_list) == 1:
            return filtered_list[0]
        else:
            return []
    elif statement_type == 'Cash Flow':
        filtered_list = [sheet for sheet in sheet_list if re.search('CF\s+\-\s+\d{1}', sheet)]
        if len(filtered_list) > 0:
            return filtered_list[0]
        else:
            return []
    #return filtered_list[0]

The create_lists function creates an exception file list, referring to companies with more than one file from the same period. This exception list will be used to help the program append two files together.

In [4]:
def create_lists(companies):
    exception_list = []
    for company in companies:
        master_file_list = glob.glob(f"C:/Users/curt.beck/Downloads/Cognaize/*{company}*.xlsx")
        special_file_list = glob.glob(f"C:/Users/curt.beck/Downloads/Cognaize/*{company}*[_][0-9].xlsx")
        for special_file in special_file_list:
            original_file = special_file[0:-7] + ".xlsx"
            if original_file in master_file_list:
                exception_list.append(original_file)
    return exception_list     

            
    

In [5]:
def get_mtd_ytd_value(file, sheet_name):
    df = pd.read_excel(file, sheet_name='Metadata')
    filtered_df = df.loc[df['Sheet name'] == sheet_name]
    val_type = filtered_df['Value column 1 period coverage'].iloc[0]
    if val_type == 'monthly':
        val_type = 'mtd'
    else:
        val_type
    return val_type
    

In [6]:
def get_mtd_ytd_value2(file, sheet_name):
    df = pd.read_excel(file, sheet_name='Metadata')
    filtered_df = df.loc[df['Sheet name'] == sheet_name]
    filtered_df = filtered_df.filter(regex="Value\s+column\s+\d{1}\s+period\s+coverage", axis=1).dropna(axis=1)
    #if val_type == 'monthly':
    #    val_type = 'mtd'
    #else:
    #    val_type
    #return val_type
    return list(filtered_df.values[0])

In [92]:
def sep_process(company, file, statement_type):
    #i = 0
    file_name = file[0:-5]
    tab_name_match = re.search("\_\d+", file).group().split('_')
    tab_name = tab_name_match[-1]
    sheet_names = pd.ExcelFile(file).sheet_names
    sheet_name = filter_sheets(sheet_names, statement_type)
    files = glob.glob(f"{file_name}[_][0-9].xlsx")
    if len(files) == 1:
        df = pd.read_excel(file, sheet_name=sheet_name)
        if df.filter(regex="YTD|ytd|Ytd|MTD|Mtd|mtd|QTD|Qtd|Months|Jan|Feb|Mar|Apr|May|Jun|June|Jul|July|Aug|Sep|September|Oct|Nov|Dec").empty:
            ytd_mtd_col = get_mtd_ytd_value2(file, sheet_name)
            i = 0
            if len(ytd_mtd_col) == len(df.columns[1:]):
                ytd_mtd_col.append("")
            for df_col in df.columns[1:]:
                df.rename(columns={df_col: str(df_col) + ' ' + str(ytd_mtd_col[i])}, inplace=True)
                i+=1
            for df_col in df.columns:
                match = re.search("\.\d{1}", df_col)
                if match is not None:
                    val = match.group()
                    replace_val  = df_col.replace(val, '')
                    df.rename(columns={df_col : replace_val}, inplace=True)
        sheet_nameslist_2 = pd.ExcelFile(files[0]).sheet_names
        sheet_name_2 = filter_sheets(sheet_nameslist_2, statement_type)
        df2 = pd.read_excel(files[0], sheet_name=sheet_name_2)
        df2 = df2.iloc[:, 1:]
        if df2.filter(regex="YTD|ytd|Ytd|MTD|Mtd|mtd|QTD|Qtd|Months|Jan|Feb|Mar|Apr|May|Jun|June|Jul|July|Aug|Sep|September|Oct|Nov|Dec").empty:
            ytd_mtd_col2 = get_mtd_ytd_value2(files[0], sheet_name_2)
            i_2 = 0
            if len(ytd_mtd_col2) != len(df2.columns):
                ytd_mtd_col2.append("")
            for df2_col in df2.columns:
                df2.rename(columns={df2_col: str(df2_col) + ' ' + str(ytd_mtd_col2[i_2])}, inplace=True)
                i_2+=1
            for df2_col in df2.columns:
                match = re.search("\.\d{1}", df2_col)
                if match is not None:
                    val = match.group()
                    replace_val  = df2_col.replace(val, '')
                    df2.rename(columns={df2_col : replace_val}, inplace=True)
        concat_df = pd.concat([df, df2], axis=1)
        
        #final_df = concat_df.T.drop_duplicates().T
        return concat_df, tab_name
    elif len(files) > 1:
        df = pd.read_excel(file, sheet_name=sheet_name)
        sheet_nameslist_2 = pd.ExcelFile(files[0]).sheet_names
        sheet_name_2 = filter_sheets(sheet_nameslist_2, statement_type)
        sheet_nameslist_3 = pd.ExcelFile(files[1]).sheet_names
        sheet_name_3 = filter_sheets(sheet_nameslist_3, statement_type)
        df2 = pd.read_excel(files[0], sheet_name=sheet_name_2)
        df2 = df2.iloc[:, 1:]
        df3 = pd.read_excel(files[-1], sheet_name=sheet_name_3)
        df3 = df3.iloc[:, 1:]
        concat_df = pd.concat([df, df2, df3], axis=1)
        for col
        #final_df = concat_df.T.drop_duplicates().T
        return concat_df, tab_name
        

In [8]:
def multiple_balance_sht(file, sheets):
    df1 = pd.read_excel(file, sheet_name=sheets[0], header=None)
    df2 = pd.read_excel(file, sheet_name=sheets[-1], header=None, skiprows=1)
    df2.dropna(inplace=True)
    concat_df = pd.concat([df1, df2], ignore_index=True)
    for col in concat_df.columns:
        concat_df.rename(columns={col: ''}, inplace=True)
    tab_name_match = re.search("\_\d{2,}", file).group().split('_')
    tab_name = tab_name_match[-1]
    return concat_df, tab_name

<li>1) Create Company List</li>
<li>2) Create Exception File List</li>
<li>3) If file is in exception list, perform separate process, otherwise perform normal process.</li>

In [93]:
#companies = ['Rocket Software']
companies = generate_company_list()
exception_list = create_lists(companies)
statement_list = ['Income Statement', 'Balance Sheet', 'Cash Flow']
#statement_type = 'Income Statement'
for statement_type in statement_list:
    for company in companies:
        if statement_type == 'Income Statement':
            file_list = glob.glob(f"C:/Users/curt.beck/Downloads/Cognaize/{company}[_]" + ('[0-9]')*6 + ".xlsx")
        else:
            file_list = glob.glob(f"C:/Users/curt.beck/Downloads/Cognaize/{company}*.xlsx")
        excel_file = pd.ExcelWriter(f"\\Users\\curt.beck\\Downloads\\Condensed_Cognaize\\{company}_{statement_type}.xlsx", engine='xlsxwriter')
        for file in file_list:
            if file in exception_list and statement_type == 'Income Statement':
                concat_df, tab_name_sep = sep_process(company, file, statement_type)
                concat_df.to_excel(excel_file, sheet_name=tab_name_sep, index=False)
            else:
                sheet_names = pd.ExcelFile(file).sheet_names
                sheet_name = filter_sheets(sheet_names, statement_type)
                if len(sheet_name)==0:
                    pass
                elif len(sheet_name) > 1 and statement_type == 'Balance Sheet' and type(sheet_name) == list:
                    concat_balance_df, tab_name = multiple_balance_sht(file, sheet_name)
                    concat_balance_df.to_excel(excel_file, sheet_name=tab_name, index=False)
                else:
                    tab_name_match = re.search("\_\d+", file).group().split('_')
                    tab_name = tab_name_match[-1]
                #sheet_name = filter_sheets(sheet_names, statement_type)
                    try:
                        df = pd.read_excel(file, sheet_name=sheet_name, header=None)
                        df.rename(columns={'Unnamed: 0': ''}, inplace=True)
                        #col_metadata = get_mtd_ytd_value2(file, sheet_name)
                        #i = 0
                        for col in df.columns:
                                df.rename(columns={col: ''}, inplace=True)
                        df.to_excel(excel_file, sheet_name=tab_name, index=False)
                    except Exception as e: 
                        print(e)
        excel_file.save()
        print(f"{company}'s {statement_type} master file has been created")

Ahlsell's Income Statement master file has been created
Dexko Global's Income Statement master file has been created
Duff & Phelps's Income Statement master file has been created
Hurtigruten's Income Statement master file has been created
Impala's Income Statement master file has been created
Informatica's Income Statement master file has been created
Rocket Software's Income Statement master file has been created
Ahlsell's Balance Sheet master file has been created
Dexko Global's Balance Sheet master file has been created
Duff & Phelps's Balance Sheet master file has been created
Hurtigruten's Balance Sheet master file has been created
Impala's Balance Sheet master file has been created
Informatica's Balance Sheet master file has been created
Rocket Software's Balance Sheet master file has been created
Ahlsell's Cash Flow master file has been created
Dexko Global's Cash Flow master file has been created
Duff & Phelps's Cash Flow master file has been created
Hurtigruten's Cash Flow mas